In [2]:
import requests
import bs4
import pickle

urls = [
    "https://www.nytimes.com/interactive/2020/01/31/us/politics/pete-buttigieg-campaign-speech.html",
    "https://www.nytimes.com/interactive/2020/01/31/us/politics/joe-biden-campaign-speech.html",
    "https://www.nytimes.com/interactive/2020/01/31/us/politics/amy-klobuchar-campaign-speech.html",
    "https://www.nytimes.com/interactive/2020/01/31/us/politics/bernie-sanders-campaign-speech.html",
    "https://www.nytimes.com/interactive/2020/01/31/us/politics/elizabeth-warren-campaign-speech.html",
    "https://www.nytimes.com/interactive/2020/01/31/us/politics/andrew-yang-campaign-speech.html"]

def url_to_transcript(url):
    r =  requests.get(url)
    soup = bs4.BeautifulSoup(r.text, "html.parser")
    content = soup.find_all('p', attrs={'class': 'g-body'})
    text = ""
    for txt in content:
        text += txt.text
    return text
    
democrats = ["Pete Buttigieg", "Joe Biden", "Amy Klobuchar", "Bernie Sanders", "Elizabeth Warren", "Andrew Yang"]


In [3]:
# Requests to transcripts
transcripts = [url_to_transcript(u) for u in urls]

In [4]:
# Pickle for later use
!mkdir transcripts
for i, c in enumerate(democrats):
    with open("transcripts/" + c + ".txt", "wb") as file:
        pickle.dump(transcripts[i], file)

A subdirectory or file transcripts already exists.


In [5]:
# Read pickle for use
data = {}
for i, c in enumerate(democrats):
    with open("transcripts/" + c + ".txt", "rb") as file:
        data[c] = pickle.load(file)

In [6]:
#Check if it's usable data
data['Joe Biden'][:]

"\n\tFormer Vice President Joseph R. Biden Jr. casts himself as an experienced hand who can appeal to black voters nationwide and win in the Midwest. On the campaign trail, he rarely delivers the same stump speech verbatim — he often goes on tangents and sometimes cuts himself off. But his core themes include rebuking President Trump’s leadership and fighting for the middle class.\n\n\tBelow is a speech he gave as tensions with Iran escalated. The speech focuses more on foreign policy than usual, but it offers a window into his message. It has been excerpted in places for clarity. Mr. Biden appeared with Representative Abby Finkenauer, Democrat of Iowa, who has endorsed him.\n\n\n\n\tI tell you what. Having, having the congresswoman from the First District go out of her way like she has for me has made a great deal of difference to me. It’s one of the most coveted endorsements anybody looks for.\n\nHey man, how you doing? Good to see you guys. I tell you what, two great friends. Anyway

In [14]:
# Combines the text
def combine_text(list_of_text):
    '''Takes a list of text and combines them into one large chunk of text.'''
    combined_text = ''.join(list_of_text)
    return combined_text

In [15]:
# 
data_combined = {key: [combine_text(value)] for (key, value) in data.items()}

In [16]:
# Data is put in a pandas dataframe, this form is called Corpus

import pandas as pd
pd.set_option('max_colwidth',150)

df = pd.DataFrame.from_dict(data_combined).transpose()
df.columns = ['transcript']
df = df.sort_index()
df

,transcript
Amy Klobuchar,\n\tThe central thrust of Senator Amy Klobuchar of Minnesota’s closing message before the Iowa caucuses can be captured in just three words: I can...
Andrew Yang,"\n\tThe businessman and entrepreneur Andrew Yang has won support by pitching himself as a numbers-driven truth teller, whose status as a political..."
Bernie Sanders,"\n\tSenator Bernie Sanders of Vermont has delivered a similar stump speech for decades, championing a vision for the country that serves working- ..."
Elizabeth Warren,"\n\tSenator Elizabeth Warren of Massachusetts and President Trump are radically different politicians, but her closing message before the Iowa cau..."
Joe Biden,\n\tFormer Vice President Joseph R. Biden Jr. casts himself as an experienced hand who can appeal to black voters nationwide and win in the Midwes...
Pete Buttigieg,"\n\tPete Buttigieg, the former mayor of South Bend, Ind., delivers a tight stump speech — usually about 12 to 15 minutes long — that is heavy on h..."


In [21]:
import re
import string

# Using regex to remove noise from the texts - should be split into two

def clean_data_1(text):
    """ Clean data part 1: Lower case,  """
    text = text.lower()
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\t', '', text)
    text = re.sub('\b', '', text)
    text = re.sub('[^a-z ]+', '', text)
    text = re.sub('\s\s', ' ', text)
    return text

round1 = lambda x: clean_data_1(x)

In [22]:
data_clean = pd.DataFrame(df.transcript.apply(round1))
data_clean


,transcript
Amy Klobuchar,the central thrust of senator amy klobuchar of minnesotas closing message before the iowa caucuses can be captured in just three words i can win w...
Andrew Yang,the businessman and entrepreneur andrew yang has won support by pitching himself as a numbersdriven truth teller whose status as a political outsi...
Bernie Sanders,senator bernie sanders of vermont has delivered a similar stump speech for decades championing a vision for the country that serves working and mi...
Elizabeth Warren,senator elizabeth warren of massachusetts and president trump are radically different politicians but her closing message before the iowa caucuses...
Joe Biden,former vice president joseph r biden jr casts himself as an experienced hand who can appeal to black voters nationwide and win in the midwest on t...
Pete Buttigieg,pete buttigieg the former mayor of south bend ind delivers a tight stump speech usually about to minutes long that is heavy on his big thematic id...


In [23]:
data_clean.transcript.loc['Andrew Yang']

my and i thought well thats the opposite of what we should have people doing so i decided to try and change itnow how many of you all work at nonprofits now how many of you have volunteered at a nonprofit you all should just raise your hand on that second one that ones pretty much are you a nice person well yes i am there was that one timeso and this is where my wife evelyn starts to come in because i went to her i was a pretty normal guy when we met i was working at a small company and i come home and im like hey i want to quit my job and start this nonprofit to help train entrepreneurs to create jobs around the countryand so how do you start a nonprofit the way i started venture for america was i founded a i put some of our savings into it and then i started calling rich friends with this question do you love america the smart among them said what does it mean if i say yes and i said at least and of them said i love america for that much i was like i thought you did so we started wit